# Gathering Data for Postfire Landsat

In [1]:
#import necessary libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import urllib

## 1. Display Data about All Wildfire Incidents in Indonesia
Gather information for all wildfire events in Sumatra

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

wf_data  = pd.read_csv("../data-total.csv")
# rename columns
wf_data.rename(
    columns={
        'Lat': 'lat', 
        'Long': 'long', 
        'Satellite': 'satellite', 
        'Time_UTC': 'time_utc', 
        'Date': 'date', 
        'Source': 'source', 
        'PROVINSI': 'provinsi',
        'KAB_KOTA': 'kab_kota',
        }, 
    inplace=True
    )

wf_data.date = pd.to_datetime(wf_data.date, format="%Y%m%d")
wf_data.date = wf_data.date.astype('string')

In [1]:
# All provinces in sumatera
sumatera_provinces = [
    'SUMATERA UTARA',
    'JAMBI',
    'RIAU',
    'SUMATERA BARAT',
    'SUMATERA SELATAN',
    'BENGKULU',
    'LAMPUNG',
    'ACEH',
    ]
# create filter so that only events in sumatera will be returned.
sumatera_filter = wf_data['provinsi'].apply(lambda x: x in sumatera_provinces) 
sumatera_only = wf_data.loc[sumatera_filter]

display(sumatera_only.info())
display(sumatera_only.provinsi.unique())
display(sumatera_only.head())

In [2]:
sumatera_only = sumatera_only.sort_values(by=['date'], 
                                          ascending=True,
                                          ignore_index=True)
sumatera_only

## 2. Download Satellite Image using Earth Engine API
Import earth engine API which is called ee

In [12]:
import ee
import folium
from folium import plugins

In [18]:
import collections
collections.Callable = collections.abc.Callable

In [3]:
# Authenticate earth engine servers
ee.Authenticate()

#initialize API
ee.Initialize()

## 3. Define Functions for Downloading Satellite Image 

In [4]:
def cloudMask(image):
    """
    :param image: satellite image
    :return: cloud-masked satellite image
    """
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20))\
                .addBands(image.metadata('system:time_start'))


def download_landsat_satellite_image(
        satellite_name, situation, 
        dataset, scale, file_format, 
        folder_path):
    """
    :param satellite_name: the satellite image name
    :param situation: condition of the wildfire, "prefire" or "postfire"
    :param dataset: dataset of the wildfire events 
    :param scale: scale for the satellite bands
    :param file_format: file format of the file being downloaded
    :param folder_path: location for the downloaded satellite image band files
    :return: None (just download the file into the specified folder_path)
    """
    
    for row in range(len(dataset)):
        file_path = os.path.join(folder_path, str(row))
        if os.path.exists(file_path):
            print("data already exist")
        else:
            os.mkdir(file_path)
        
        os.chdir(file_path)
        print(os.getcwd())
        # Set coordinates
        print(f"row {row} requesting...")
        coordinates = [float(dataset.lat.values[row]), float(dataset.long.values[row])]
        region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                        coordinates[1]+0.2, coordinates[0]+0.2]);


        # filter image collection
        # set base date
        base_date = ee.Date(dataset.date.values[row])

        # set geometry point
        point = ee.Geometry.Point(dataset.long.values[row], dataset.lat.values[row])

        # pre fire
        prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
        prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

        # landsat image pre wildfire event
        prefire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(prefire_date_start, prefire_date_end)\
                                   .filterBounds(region)\
                                   .map(cloudMask)\
                                   .median()

        # Calculate NBR for pre fire
        pre_nbr = prefire_collection.normalizedDifference(['B5', 'B7'])


        # post fire
        postfire_date_start = ee.Date(base_date.advance(1, 'day'))
        postfire_date_end = ee.Date(base_date.advance(365, 'day'))

        # sentinel image post wildfire event
        postfire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(postfire_date_start, postfire_date_end)\
                                   .filterBounds(region)\
                                   .map(cloudMask)\
                                   .median()

        # Calculate NBR for post fire
        post_nbr = postfire_collection.normalizedDifference(['B5', 'B7'])

        # calculate delta NBR
        delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
        postfire_collection = postfire_collection.addBands(delta_nbr)
        
        if situation == 'prefire':
            filenames= ['b7_b5','b4_b3','b2']
            bands =  [
                ['B7', 'B5'], 
                ['B4', 'B3'], 
                ['B2'],
                ]
            
            for i in range(len(filenames)):
                if filenames[i] not in os.listdir():
                    download_url = prefire_collection.getDownloadURL({
                        'region': region,
                        'bands' : bands[i],
                        'scale' : scale,
                        'format': file_format
                    })
                    
                    print(f"download data from {download_url}")
                    urllib.request.urlretrieve(download_url, filenames[i])
                    print(f"{download_url} downloaded")
                
                else:
                    print(f"{bands[i]} already exist")
                    
        else:
            filenames= ['dnbr_b7','b5_b4','b3_b2']
            bands =  [
                ['DELTA_NBR', 'B7'], 
                ['B5', 'B4'], 
                ['B3', 'B2'],
                ]

            for i in range(len(filenames)):
                if filenames[i] not in os.listdir():
                    download_url = postfire_collection.getDownloadURL({
                        'region': region,
                        'bands' : bands[i],
                        'scale' : scale,
                        'format': file_format
                    })

                    print(f"download data from {download_url}")
                    urllib.request.urlretrieve(download_url, filenames[i])
                    print(f"{download_url} downloaded")
                else:
                    print(f"{bands[i]} already exist")

## 4. Create Folder to Store Satellite Image

In [5]:
D_DIR = "D:\\"

WILDFIRE_SUMATERA_DATASET_DIR = "wildfire-sumatera-dataset"
WILDFIRE_SUMATERA_GEOTIFF_DIR = "wildfire-sumatera-geotiff"

LANDSAT_DIR = "landsat-8"

LANDSAT_8_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{LANDSAT_DIR}"
LANDSAT_PREFIRE_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{LANDSAT_DIR}\\prefire"
LANDSAT_POSTFIRE_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{LANDSAT_DIR}\\postfire"

if D_DIR is not os.getcwd():
    os.chdir(D_DIR)
    print(f"OS directory is changed to {D_DIR}")
    print(os.getcwd())
    print()
    
if WILDFIRE_SUMATERA_DATASET_DIR not in os.listdir():
    os.mkdir(WILDFIRE_SUMATERA_DATASET_DIR)
    print(f"{WILDFIRE_SUMATERA_DATASET_DIR} has been created")
else:
    print(f"{WILDFIRE_SUMATERA_DATASET_DIR} already exist")


In [6]:
# check whether the folder is exists
if not os.path.exists(LANDSAT_PREFIRE_FOLDER):
    os.makedirs(LANDSAT_PREFIRE_FOLDER)
    print(f"{LANDSAT_PREFIRE_FOLDER} has been successfully create")
else:
    print(f"{LANDSAT_PREFIRE_FOLDER} already exist")

if not os.path.exists(LANDSAT_POSTFIRE_FOLDER):
    os.makedirs(LANDSAT_POSTFIRE_FOLDER)
    print(f"{LANDSAT_POSTFIRE_FOLDER} has been successfully create")
else:
    print(f"{LANDSAT_PREFIRE_FOLDER} already exist")

## 5. Download the Satellite Image

In [7]:
download_landsat_satellite_image(
    satellite_name='LANDSAT/LC08/C02/T1_TOA', 
    situation='postfire',                                                                     
    dataset=sumatera_only, 
    scale=50, 
    file_format='GeoTIFF',
    folder_path=LANDSAT_POSTFIRE_FOLDER,
    )

print("done")